In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import shutil
import cv2
import sys
from tqdm import tqdm

import wave
import pylab
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
import itertools

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from keras.layers import Add, Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D , Dropout , Concatenate, Reshape , Lambda
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Bidirectional,LSTM,Reshape
from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras.metrics import Recall,Precision
from keras.callbacks import ModelCheckpoint
from keras.initializers import glorot_uniform
from keras import backend as K

from keras.applications.vgg16 import VGG16

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import extract_data
import create_multimodal_pairs
import architectures
import siamese

In [ ]:
# Declare constants

BATCH_SIZE = 64
N_CHANNELS = 3
N_CLASSES = 20

IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMG_SHAPE = (IMAGE_HEIGHT,IMAGE_WIDTH,N_CHANNELS)

AUD_HEIGHT = 128
AUD_WIDTH = 128
AUD_SHAPE = (AUD_HEIGHT,AUD_WIDTH,N_CHANNELS)

In [ ]:
data_dir =  #set path to the folder containing subfolders containing your images 

#list of subfolders which denote different classes of your dataset
categories = os.listdir(data_dir)

data_img = []
categorical_index_img = [] #to keep track of which class is mapped to which index

data_img,categorical_index_img = extract_data.create_data(data_dir,categories,data_img,categorical_index_img,IMAGE_HEIGHT,IMAGE_WIDTH)


In [ ]:
data_dir =#set path to the folder containing subfolders containing the audio spectogram images

#list of subfolders which denote different classes of your dataset
categories = os.listdir(data_dir)

data_aud = []
categorical_index_aud = []

data_aud,categorical_index_aud = extract_data.create_data(data_dir,categories,data_aud,categorical_index_aud,AUD_HEIGHT,AUD_WIDTH)

In [ ]:
import random

random.seed(33)
random.shuffle(data_img)
random.shuffle(data_aud)

In [ ]:
# creating X and y from data list

X_img,y_img = extract_data.extract_features_and_labels(data_img)
X_aud,y_aud = extract_data.extract_features_and_labels(data_aud)

In [ ]:
from sklearn.model_selection import train_test_split

# segregating the training and test image data
X_train_img,X_test_img,y_train_img,y_test_img = train_test_split(X_img,y_img,test_size=0.1) # 0.1 test to train split 

# segregating the training and test audio data
X_train_aud,X_test_aud,y_train_aud,y_test_aud = train_test_split(X_aud,y_aud,test_size=0.1) # 0.1 test to train split 

Shape of X_train:(587, 128, 128, 3)
Shape of y_train:(587,)
Shape of X_test:(66, 128, 128, 3)
Shape of y_test:(66,)


In [ ]:
# create pairs from training data

# to define the same list of classes
same_class_list1 = []
same_class_list2 = []
img_a_train,aud_a_train,img_b_train,aud_b_train,labels_train  = create_multimodal_pairs.multimodal_data_generate(X_train_img , y_train_img , X_train_aud , y_train_aud)

In [ ]:
# create pairs from test data

# to define the same list of classes
same_class_list1 = []
same_class_list2 = [] 
img_a_test,aud_a_test,img_b_test,aud_b_test,labels_test  = create_multimodal_pairs.multimodal_data_generate(X_test_img , y_test_img , X_test_aud , y_test_aud)

In [ ]:
# randomize the train data

img_a_train,aud_a_train,img_b_train,aud_b_train,labels_train = extract_data.randomize_zip(img_a_train,aud_a_train,img_b_train,aud_b_train,labels_train)

In [ ]:
# randomize the test data

img_a_test,aud_a_test,img_b_test,aud_b_test,labels_test = extract_data.randomize_zip(img_a_test,aud_a_test,img_b_test,aud_b_test,labels_test )

In [ ]:
input_dim_img = (IMAGE_HEIGHT,IMAGE_WIDTH,N_CHANNELS)
input_dim_aud = (AUD_HEIGHT,AUD_WIDTH,N_CHANNELS)

# final model train 
optimizer , model = siamese.siamese_network(input_dim_img , input_dim_aud)


In [ ]:
weight_dir = #set a weights directory
if not os.path.exists(weight_dir):
    os.mkdir(weight_dir)
    
#checkpoints for best weights in terms of validation accuracy
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=weight_dir+'/checkpoint-{epoch:02d}-{val_loss:.4f}.hdf5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
model.fit([np.asarray(img_a_train),np.asarray(aud_a_train),np.asarray(img_b_train),np.asarray(aud_b_train)], np.asarray(labels_train) ,validation_split=0.2, batch_size = BATCH_SIZE , verbose = 1, epochs = 30 , callbacks = checkpoint)